# driver_pareto_hr
Generate Pareto front plot for LS-ROM and NM-ROM by varying number of HR nodes.  

In [ ]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sp
import matplotlib.pyplot as plt
from matplotlib import cm
import dill as pickle
from time import time
import sys, os
from matplotlib.ticker import StrMethodFormatter, NullFormatter
from utils.Burgers2D_probgen import Burgers2D
from utils.domain_decomposition import DD_model
from utils.LS_ROM import compute_bases_from_svd, DD_LS_ROM
from utils.NM_ROM import DD_NM_ROM, separate_snapshots, RBFmdl

In [ ]:
data_dir = './data/'
plt.rc('font', size=20)
plt.rcParams['text.usetex'] = True

# Build full-domain FOM and DD-FOM

In [ ]:
# define constant parameters for PDE
nx, ny  = 480, 24
x_lim   = [-1.0, 1.0]
y_lim   = [0.0, 0.05]

na1, nlam = 80, 80
a1_lim  = [1.0, 10000.0]
lam_lim = [5.0, 25.0]
# A, Lam  = np.meshgrid(np.linspace(a1_lim[0], a1_lim[1], na1), np.linspace(lam_lim[0], lam_lim[1], nlam))
# Mu      = np.vstack([A.flatten(), Lam.flatten()]).T

a1, lam = 7692.5384, 21.9230
viscosity = 1e-1

# number of subdomains in x and y directions for DD model
n_sub_x = 2
n_sub_y = 2

n_sub   = n_sub_x*n_sub_y

intr_size = 8
intf_size = 4
act_type = 'Swish'
nhr_list = [50, 100, 150, 200, 250]

In [ ]:
# store directory names for figures
fig_dir  = f'./figures/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
svd_dir  = f'./data/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
ls_figs = fig_dir + 'lsrom/'
nm_figs = fig_dir + 'nmrom/'
for d in [fig_dir, ls_figs, nm_figs]:
    if not os.path.exists(d):
        os.mkdir(d)

In [ ]:
# load snapshot and residual data
file = f'./data/residual_nx_{nx}_ny_{ny}_mu_{viscosity}_Nsamples_400.p'
data = pickle.load(open(file, 'rb'))
Mu = data['parameters']
snapshots = data['snapshots']
residuals = data['residuals']

# load snapshot and residual data
# file = f'./data/snapshot_nx_{nx}_ny_{ny}_mu_{viscosity}_Nsamples_6400.p'
# data = pickle.load(open(file, 'rb'))
# Mu = data['parameters']
# snapshots = data['snapshots']

In [ ]:
# compute FOM for given a1 and lambda
def u_exact(x, y):
    phi = a1 + a1*x + (np.exp(lam*(x-1.0)) + np.exp(-lam*(x - 1.0)))*np.cos(lam*y)
    val = -2.0*viscosity*(a1 + lam*(np.exp(lam*(x-1.0)) \
          - np.exp(-lam*(x - 1.0)))*np.cos(lam*y))/phi
    return val
def v_exact(x, y):
    phi = a1 + a1*x + (np.exp(lam*(x-1.0)) + np.exp(-lam*(x - 1.0)))*np.cos(lam*y)
    val =  2.0*viscosity*(lam*(np.exp(lam*(x-1.0)) \
           + np.exp(-lam*(x - 1.0)))*np.sin(lam*y))/phi
    return val

# generate Burgers FOM on full domain
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity, u_exact, v_exact)
print('Solving full domain model:')
sys.stdout.flush()
u_fom, v_fom, res_hist = fom.solve(np.zeros(fom.nxy), np.zeros(fom.nxy), tol=1e-8, print_hist=False)
sol_fd = np.concatenate([u_fom, v_fom])

# compute Burgers DD FOM
ddmdl = DD_model(fom, n_sub_x, n_sub_y)
ndof_fom = 2*np.sum([s.n_interior+s.n_interface for s in ddmdl.subdomain]) \
            + ddmdl.subdomain[0].constraint_mat.shape[0]
w0 = np.zeros(ndof_fom)
print('\nSolving DD model:')
sys.stdout.flush()
u_dd_fom, v_dd_fom, u_intr, v_intr, u_intf, v_intf, lam_fom, fom_time, itr = ddmdl.solve(w0, tol=1e-9, 
                                                                           maxit=15,
                                                                           print_hist=True)
sol_dd_fom = np.concatenate([u_dd_fom, v_dd_fom])

dd_fom_rel_err = np.linalg.norm(sol_dd_fom-sol_fd)/np.linalg.norm(sol_fd)
print(f'\nDD-FOM relative error = {dd_fom_rel_err:1.4e}')
sys.stdout.flush()

## LS-ROM

In [ ]:
# energy criteria for number of residual basis vectors
ec_res  = 1e-10

# number of interior and interface basis vectors
nbasis_res = -1
nbasis_intr = 8
nbasis_intf = 4
    
# load SVD data
Ntotal = 6400 if nx in [240,480] else 4200
print('Loading SVD data...')
intr_dict = pickle.load(open(svd_dir+f'intr_svd_nsnaps_{Ntotal}.p', 'rb'))
intf_dict = pickle.load(open(svd_dir+f'intf_svd_nsnaps_{Ntotal}.p', 'rb'))
res_dict  = pickle.load(open(svd_dir+'res_svd.p', 'rb'))
print('Data loaded!')

# compute bases
print('Computing bases...')
interior_bases = compute_bases_from_svd(intr_dict, nbasis=intr_size)
interface_bases = compute_bases_from_svd(intf_dict, nbasis=intf_size)
residual_bases = compute_bases_from_svd(res_dict, ec=ec_res, nbasis=nbasis_res)
print('Bases computed!')

for i in range(n_sub):
    print(f'residual_bases[{i}]={residual_bases[i].shape}')
for i in range(n_sub):
    print(f'interior_bases[{i}]={interior_bases[i].shape}') 
for i in range(n_sub):
    print(f'interface_bases[{i}]={interface_bases[i].shape}')    
sys.stdout.flush()
sys.stdout.flush()

In [ ]:
lsrom_rel_times = np.zeros(len(nhr_list))
lsrom_rel_errors = np.zeros(len(nhr_list))

for j, nhr in enumerate(nhr_list):

    # compute DD LS-ROM
    print('Building DD LS-ROM...')
    print(f'# HR nodes = {nhr}')
    sys.stdout.flush()
    ddlsrom = DD_LS_ROM(ddmdl, residual_bases, interior_bases, interface_bases, 
                        hr=True,
                        hr_type='collocation',
                        n_samples=nhr,
                        n_corners=30,
                        constraint_type='weak', 
                        n_constraints=1, 
                        seed=2)
    print('ROM built!\n')
    sys.stdout.flush()

    # solve DD ROM
    print('Solving DD LS-ROM...')
    sys.stdout.flush()

    ndof_rom = np.sum([s.n_interior+s.n_interface for s in ddlsrom.subdomain]) \
                + ddlsrom.n_constraints
    w0 = np.zeros(ndof_rom)
    u_dd_rom, v_dd_rom, w_intr, w_intf, lam_ls, rom_time, itr = ddlsrom.solve(w0, tol=1e-4, maxit=15, print_hist=True)
    print('Solution found!\n')
    sys.stdout.flush()

    # compute error
    dd_rom_rel_err = ddlsrom.compute_error(w_intr, w_intf, u_intr, v_intr, u_intf, v_intf)
    print(f'DD-ROM rel. error = {dd_rom_rel_err:1.4e}')
    print(f'Speedup = {fom_time/rom_time}\n')
    sys.stdout.flush()
    
    lsrom_rel_times[j]  = rom_time/fom_time
    lsrom_rel_errors[j] = dd_rom_rel_err

## NM-ROM

In [ ]:
# restrict snapshots to interior and interface subdomains
interior, interface = separate_snapshots(ddmdl, snapshots)

In [ ]:
net_folder0 = f'./trained_nets/nx_{nx}_ny_{ny}_mu_{viscosity}_{n_sub_x}x_by_{n_sub_y}y/'
Ntotal = 6400 if nx in [240, 480] else 4200
intr_rnnz = 5
intf_rnnz = 5
intr_shift = 5
intf_shift = 5

intr_net_list = [net_folder0+\
                 f'sub_{i+1}of{n_sub}/interior/'+\
                 f'ld_{intr_size}_rnnz_{intr_rnnz}_rshift_{intr_shift}_{act_type}_batch_32_AbsMSEloss_{Ntotal}snaps.p' \
                  for i in range(n_sub)]
intf_net_list = [net_folder0+\
                 f'sub_{i+1}of{n_sub}/interface/'+\
                 f'ld_{intf_size}_rnnz_{intf_rnnz}_rshift_{intf_shift}_{act_type}_batch_32_AbsMSEloss_{Ntotal}snaps.p' \
                  for i in range(n_sub)]

nmrom_rel_times = np.zeros(len(nhr_list))
nmrom_rel_errors = np.zeros(len(nhr_list))

for j, nhr in enumerate(nhr_list):
        
    # compute DD -ROM
    print('Building DD NM-ROM...')
    print(f'# HR nodes = {nhr}')
    sys.stdout.flush()
    ddnmrom = DD_NM_ROM(ddmdl, 
                        intr_net_list,
                        intf_net_list,
                        residual_bases,
                        hr=True, 
                        n_samples=nhr, 
                        n_corners=30, 
                        constraint_type='weak', 
                        n_constraints=1,
                        seed=2)
    print('ROM built!\n')
    sys.stdout.flush()
    
    print('Computing RBF interpolant...')
    rbfmdl = RBFmdl(ddnmrom, Mu, interior, interface, neighbors=None, kernel='linear')
    print('Interpolant computed!')

    # solve DD ROM
    print('Solving DD NM-ROM...')
    sys.stdout.flush()

    ndof_rom = np.sum([s.intr_latent_dim+s.intf_latent_dim for s in ddnmrom.subdomain]) \
                       + ddnmrom.n_constraints
#     w0 = np.zeros(ndof_rom)

    w0, lam0, initial_time = rbfmdl.get_initial(np.array([[a1, lam]]), ddnmrom)
    w0 = np.concatenate([w0, lam0])
    
    u_dd_rom, v_dd_rom, w_intr, w_intf, lam_nm, rom_time, itr = ddnmrom.solve(w0, tol=1e-4, maxit=15, print_hist=True)
    rom_time += initial_time
    print('Solution found!\n')
    sys.stdout.flush()

    # compute error
    dd_rom_rel_err = ddnmrom.compute_error(w_intr, w_intf, u_intr, v_intr, u_intf, v_intf)
    print(f'DD NM-ROM rel. error = {dd_rom_rel_err:1.4e}')
    print(f'Speedup = {fom_time/rom_time}\n')
    sys.stdout.flush()
    
    nmrom_rel_times[j]  = rom_time/fom_time
    nmrom_rel_errors[j] = dd_rom_rel_err
    

In [ ]:
# plot pareto fronts
fig, ax = plt.subplots(figsize=(10, 8))

ax.scatter(lsrom_rel_times, lsrom_rel_errors, s=200, c='blue', marker='o', label='LS-ROM', alpha=0.75)
ax.scatter(nmrom_rel_times, nmrom_rel_errors, s=200, c='red', marker='^', label='NM-ROM', alpha=0.75)

ls_xlabel = [1.03, 0.89, 1.01, 0.87, 0.89]
ls_ylabel = [1.01, 0.89, 1.01, 0.89, 0.89] 
nm_xlabel = [1.01, 0.89, 1.01, 0.86, 0.90]
nm_ylabel = [1.01, 0.89, 1.01, 0.92, 0.87] 

for i, nhr in enumerate(nhr_list):
    ax.text(lsrom_rel_times[i]*ls_xlabel[i], lsrom_rel_errors[i]*ls_ylabel[i], nhr)
    ax.text(nmrom_rel_times[i]*nm_xlabel[i], nmrom_rel_errors[i]*nm_ylabel[i], nhr)

front_x = np.concatenate((lsrom_rel_times[[0, 1, 3, 4]], nmrom_rel_times[[1, 3, 4]]))
front_y = np.concatenate((lsrom_rel_errors[[0, 1, 3, 4]], nmrom_rel_errors[[1, 3, 4]]))

ax.plot(front_x, front_y, 'k.-', markersize=12, linewidth=2, label='Pareto front')

ax.set_yscale('log')
ax.set_xscale('log')

xlim = [min(lsrom_rel_times.min(), nmrom_rel_times.min())*0.85, 
        max(lsrom_rel_times.max(), nmrom_rel_times.max())*1.15]
ylim = [min(lsrom_rel_errors.min(), nmrom_rel_errors.min())*0.85,
        max(lsrom_rel_errors.max(), nmrom_rel_errors.max())*1.15]

xtick_locs = np.logspace(np.log10(xlim[0]), np.log10(xlim[1]), 5)
xtick_labels = [f'{v:1.1e}' for v in xtick_locs]
ytick_locs = np.logspace(np.log10(ylim[0]), np.log10(ylim[1]), 5)
ytick_labels = [f'{v:1.1e}' for v in ytick_locs]

ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.xaxis.set_minor_formatter(NullFormatter())
ax.set_xticks(xtick_locs, xtick_labels)
ax.set_yticks(ytick_locs, ytick_labels)

ax.set_xlabel('Relative wall time')
ax.set_ylabel('Relative error')
ax.grid()
ax.legend(loc='lower left')
file = fig_dir+f'pareto_hr.png'
plt.savefig(file, bbox_inches='tight', pad_inches=0.1)